In [1]:
import matplotlib.pyplot as plt

import networkx as nx

import numpy as np

# For reading files
import os

# Data manipulation
import pandas as pd

# Text cleaning
import re

import requests

## Text processing
# NLP
import spacy
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load("en_core_web_sm")

# Sentence boundary detection
import pysbd

# More NLP for fun
from textblob import TextBlob

from tqdm import tqdm

# Pipeline creation
from sklearn.pipeline import Pipeline

# Text extraction
from tika import parser

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [2]:
# Wanted to see details
pd.options.display.max_rows = 999

## Get the cover letter df did

In [6]:

# read_file = pd.read_excel (r"C:\Users\sambe\Projects\Cover_Letter_Analysis\data\Data_tobecompiled\Media_Planner_New York_NY_2020-06-10\Cover_Letter\Media_Planner_New York_NY_2020-06-10 2.csv")
# read_file.to_csv ('cover_letters.csv', index = None, header=True)
cover_letter = pd.read_csv(r'C:\Users\sambe\Projects\Cover_Letter_Analysis\data\Data_tobecompiled\Media_Planner_New York_NY_2020-06-10\Cover_Letter\Media_Planner_New York_NY_2020-06-10 2.csv')

In [46]:
# Pull the data from the numberous cover letter files. 

more_cover_letter = pd.DataFrame(columns=['username', 'raw_cover_letters'])

all_text = []
filenames = []

def extract_data():
    for filename in os.listdir(r'C:\Users\sambe\Projects\Cover_Letter_Analysis\data\Data_tobecompiled\Media_Planner_New York_NY_2020-06-10\Cover_Letter\Add.Cover_Letters'):
        parsed = parser.from_file(r'C:\Users\sambe\Projects\Cover_Letter_Analysis\data\Data_tobecompiled\Media_Planner_New York_NY_2020-06-10\Cover_Letter\Add.Cover_Letters\\' + filename)
        text = parsed['content']
        if parsed['content'] == None:
            print(filename + " is bad.")
        filenames.append(filename)
#         resumes_df.username = filenames
        try:
            text = text.replace('\n', '')          
        except:
            pass
        all_text.append(text)
#         print("extracting data from " + filename)
    more_cover_letter.raw_cover_letters = all_text
    more_cover_letter.username = filenames
#     resumes_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
    
extract_data()

In [52]:
more_cover_letter = pd.read_csv('more_cover_letter.csv')

In [26]:
cover_letter.to_csv('cover_letter.csv')

In [29]:
cover_letter = pd.read_csv('cover_letter.csv', index_col=0)

In [48]:
more_cover_letter = more_cover_letter.replace(to_replace = '.pdf', value = '', regex = True)
more_cover_letter = more_cover_letter.replace(to_replace = '.docx', value = '', regex = True)
more_cover_letter = more_cover_letter.replace(to_replace = ' Letter of Recommendation.pdf', value = '', regex = True)

In [49]:
more_cover_letter['username'] = more_cover_letter['username'].str.replace(r"([A-Z])", r" \1").str.strip()

In [8]:
# Drops extra columns. We only want username and text columns
cover_letter = cover_letter.drop(cover_letter.columns[2:5], axis=1)

In [51]:
more_cover_letter.to_csv('more_cover_letter.csv')

In [56]:
# Drops first two rows, which is usually NaN and the date. 
# Renames the columns to username and raw_cover_letters
# Orders the usernames alphabetically and resets the index

# cover_letter = cover_letter.drop([0,1], axis=0)
# cover_letter.columns = ['username', 'raw_cover_letters']
cover_letter = cover_letter.sort_values('username').reset_index(drop=True)

In [10]:
# Creates a column in between for the title of the position they applied for. Cleans the text by deleting the \n

cover_letter.insert(1, 'title', 'Media Planner')
cover_letter = cover_letter.replace(to_replace = '\n', value = '', regex = True)

In [25]:
# Check to see the completed df all clean!

cover_letter.username[-1]

'Kendra Brown'

In [53]:
cover_letter

,username,title,raw_cover_letters
0,Aakash Patel,Media Planner,"I was excited to discover the Media Planner position with Rookieplay, which I saw posted on Indeed. Rookieplay is a company on the rise, and as a recent college graduate with 2 years of diverse ma..."
1,Aaron Neal,Media Planner,"Dear Hiring Manager,I am writing to you today to apply for the open position of Media Planner. As a marketing professional with 2+ years of experience I believe that I will be an asset to the Rook..."
2,Aaron Penzel,Media Planner,"Dear Hiring Manager,I am writing to express interest in Rookieplay’s Media Planner position. I am a creative and out-of-the-box marketer in the entertainment industry, with experience in both non-..."
3,Abigail Nelson,Media Planner,"To Whom It Concerns,I am a recent college graduate with a B.A. in Advertising and aminor in Psychology from Iowa State University. I have a stronginterest in advertising, specifically media planni..."
4,Abraham Jauregui,Media Planner,"June 7th, 2020RookieplayNew York, New YorkDear Hiring Professional,Please accept this letter as an expression of my interest in the Media Planner position. I am a highly motivated and progress-foc..."
...,...,...,...
121,Vickent Vera,Media Planner,"Vickent VeraNew York, NY(917)3918411vickentvera@gmail.comGreetings,I’ve been exposed and nurtured in movies and the internet from a very young age. Creating content like that has been my primary c..."
122,Vincent Gray,Media Planner,"Dear Hiring Manager:Please accept this letter and the accompanying resume as my expressed interest in the position of Media Planner currently available at your organization.As my resume reflects, ..."
123,Wania Nawaz,Media Planner,"I was motivated to apply to this internship because throughout my academic years I have fallen in love with marketing, specifically in social media marketing. My goal in the future would be workin..."
124,William Day,Media Planner,My name is William Day and I am writing to express my greatest interest in becoming a member of the Rookeiplay team. I hope that you will see that my combined experience in multiple facets of the ...


In [55]:
cover_letter = cover_letter.append(more_cover_letter)

In [58]:
cover_letter.drop_duplicates(subset ="username", 
                     keep = 'first', inplace = True) 

In [59]:
len(cover_letter)

126

## Get that resume df did

In [61]:
# Construct blank dataframe with the columns needed
resumes_df = pd.DataFrame(columns=['username', 'raw_resume'])
resumes_df

all_text = []
filenames = []




# Pull the data from the numberous resume files. 

def extract_data():
    directory = r"C:\Users\sambe\Projects\Cover_Letter_Analysis\data\Data_tobecompiled\Media_Planner_New York_NY_2020-06-10"
    for filename in os.listdir(directory):
        parsed = parser.from_file(directory + "\\" + filename)
        text = parsed['content']
        if parsed['content'] == None:
            print(filename + " is bad.")
        filenames.append(filename)
#         resumes_df.username = filenames
        try:
            text = text.replace('\n', '')          
        except:
            pass
        all_text.append(text)
#         print("extracting data from " + filename)
    resumes_df.raw_resume = all_text
    resumes_df.username = filenames
#     resumes_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
    
extract_data()

In [62]:
len(all_text)

334

In [63]:
resumes_df

,username,raw_resume
0,ResumeAakashPatel.pdf,"Microsoft Word - Aakash Patel Resume 1.docx\tAakash Patel aakashp047@gmail.com v (516) 404-6119 v Syosset, NY EDUCATION Stony Brook University ..."
1,ResumeAaronNeal.pdf,"Resume_Aaron Neal_052020_v2A A R O N M . N E A LSTRATEGISTPERSONALTRAITS & SKILLSEDUCATIONCONTACT770.871.5294aaronm.neal3@gmail.comProficient in Simmons OneView, Global Web Index MS Office Persp..."
2,ResumeAaronPenzel.pdf,"AARON PENZELAARON PENZELENTERTAINMENT ADVERTISERCONTACTPHONE: (908) 217-7289 EMAIL: A.PENZEL724@GMAIL.COM 23-43 28TH STREET LONG ISLAND CITY, NY 11105 WORK EXPERIENCEManaged all traditional media..."
3,ResumeAbbyCleary.pdf,"Indeed ResumeAbby ClearyBuffalo, NYabbycleary7_pb3@indeedemail.com(716) 238-0440Authorized to work in the US for any employerWork ExperienceInternSwanson Communications - New York, NYJune 2019 to ..."
4,ResumeAbigailNelson.pdf,"White and Pastel Teal Minimalist Resume A B I G A I L N E L S O N (651) 408-2139 | aanelson@iastate.edu | 10849 Andes Circle, IGH, MN 55077Portfolio: https://aanelson.wixsite.com/abigailnelsonLin..."
...,...,...
329,ResumeWilliamDay.pdf,"Indeed ResumeWilliam DayThe ""Jack of all Trades"" When it Comes to MediaMiami Beach, FL 33140williamday846_ys9@indeedemail.com770-687-4809It may be unorthodox to say this but I feel that to best un..."
330,ResumeYasminaOuedraogo.pdf,"Indeed ResumeYasmina OuedraogoBrooklyn, NY 11207yasminaouedraogo9_k5n@indeedemail.com3478630020Work ExperienceInternNew York Stage Supreme Court - Brooklyn, NYJuly 2019 to August 2019Observed cour..."
331,ResumeYayonahBangura.docx,YAYONAH BANGURAPage Two of TwoYAYONAH BANGURA (732) 421-4186 Yayonahb@gmail.com www.linkedin.com/in/yayonahbWebsite: theaknown.com/features-1PROFILEAmbitious and motivated communications special...
332,ResumeZacharyBrooks.pdf,"Zachary H. Brooks 2855 Nicole Ct., Oceanside, New York, 11572 516-761-8168 | Zachary.Brooks@quinnipiac.edu EDUCATION QUINNIPIAC UNIVERSITY | School of Communications ..."


In [137]:
# Drops the first row, which is usually the spreadsheet of cover letters

resumes_df = resumes_df.drop(resumes_df.index[0])

In [64]:
# Cleans the dataframe of useless drivel including file extensions, Indeed, numbers. Feel free to scan through the 
# usernames and add more!

resumes_df = resumes_df.replace(to_replace = 'Resume', value = '', regex = True)
resumes_df = resumes_df.replace(to_replace = 'Indeed', value = '', regex = True)
resumes_df = resumes_df.replace(to_replace = '.docx', value = '', regex = True)
resumes_df = resumes_df.replace(to_replace = ' \(1\)', value = '', regex = True)
resumes_df = resumes_df.replace(to_replace = '.pdf', value = '', regex = True)
resumes_df = resumes_df.replace(to_replace = '.doc', value = '', regex = True)
resumes_df['username'] = resumes_df['username'].str.replace(r"([A-Z])", r" \1").str.strip()

In [65]:
resumes_df

,username,raw_resume
0,Aakash Patel,"Microsoft Word - Aakash Patel 1\tAakash Patel aakashp047@gmail.com v (516) 404-6119 v Syosset, NY EDUCATION Stony Brook University ..."
1,Aaron Neal,"_Aaron Neal_052020_v2A A R O N M . N E A LSTRATEGISTPERSONALTRAITS & SKILLSEDUCATIONCONTACT770.871.5294aaronm.neal3@gmail.comProficient in Simmons OneView, Global Web Index MS Office Perspective..."
2,Aaron Penzel,"AARON PENZELAARON PENZELENTERTAINMENT ADVERTISERCONTACTPHONE: (908) 217-7289 EMAIL: A.PENZEL724@GMAIL.COM 23-43 28TH STREET LONG ISLAND CITY, NY 11105 WORK EXPERIENCEManaged all traditional media..."
3,Abby Cleary,"Abby ClearyBuffalo, NYabbycleary7_pb3@indeedemail.com(716) 238-0440Authorized to work in the US for any employerWork ExperienceInternSwanson Communications - New York, NYJune 2019 to Present• Ass..."
4,Abigail Nelson,"White and Pastel Teal Minimalist A B I G A I L N E L S O N (651) 408-2139 | aanelson@iastate.edu | 10849 Andes Circle, IGH, MN 55077Portfolio: https://aanelson.wixsite.com/abigailnelsonLinkedIn:..."
...,...,...
329,William Day,"William DayThe ""Jack of all Trades"" When it Comes to MediaMiami Beach, FL 33140williamday846_ys9@indeedemail.com770-687-4809It may be unorthodox to say this but I feel that to best understand who..."
330,Yasmina Ouedraogo,"Yasmina OuedraogoBrooklyn, NY 11207yasminaouedraogo9_k5n@indeedemail.com3478630020Work ExperienceInternNew York Stage Supreme Court - Brooklyn, NYJuly 2019 to August 2019Observed court proceeding..."
331,Yayonah Bangura,YAYONAH BANGURAPage Two of TwoYAYONAH BANGURA (732) 421-4186 Yayonahb@gmail.com www.linkedin.com/in/yayonahbWebsite: theaknown.com/features-1PROFILEAmbitious and motivated communications special...
332,Zachary Brooks,"Zachary H. Brooks 2855 Nicole Ct., Oceanside, New York, 11572 516-761-8168 | Zachary.Brooks@quinnipiac.edu EDUCATION QUINNIPIAC UNIVERSITY | School of Communications ..."


In [66]:
# Save as csv to be further edited in google sheets.

resumes_df.to_csv('resumes.csv')

In [67]:
resumes = pd.read_csv('resumes.csv', index_col=0)

In [68]:
resumes

,username,raw_resume
0,Aakash Patel,"Microsoft Word - Aakash Patel 1\tAakash Patel aakashp047@gmail.com v (516) 404-6119 v Syosset, NY EDUCATION Stony Brook University ..."
1,Aaron Neal,"_Aaron Neal_052020_v2A A R O N M . N E A LSTRATEGISTPERSONALTRAITS & SKILLSEDUCATIONCONTACT770.871.5294aaronm.neal3@gmail.comProficient in Simmons OneView, Global Web Index MS Office Perspective..."
2,Aaron Penzel,"AARON PENZELAARON PENZELENTERTAINMENT ADVERTISERCONTACTPHONE: (908) 217-7289 EMAIL: A.PENZEL724@GMAIL.COM 23-43 28TH STREET LONG ISLAND CITY, NY 11105 WORK EXPERIENCEManaged all traditional media..."
3,Abby Cleary,"Abby ClearyBuffalo, NYabbycleary7_pb3@indeedemail.com(716) 238-0440Authorized to work in the US for any employerWork ExperienceInternSwanson Communications - New York, NYJune 2019 to Present• Ass..."
4,Abigail Nelson,"White and Pastel Teal Minimalist A B I G A I L N E L S O N (651) 408-2139 | aanelson@iastate.edu | 10849 Andes Circle, IGH, MN 55077Portfolio: https://aanelson.wixsite.com/abigailnelsonLinkedIn:..."
...,...,...
329,William Day,"William DayThe ""Jack of all Trades"" When it Comes to MediaMiami Beach, FL 33140williamday846_ys9@indeedemail.com770-687-4809It may be unorthodox to say this but I feel that to best understand who..."
330,Yasmina Ouedraogo,"Yasmina OuedraogoBrooklyn, NY 11207yasminaouedraogo9_k5n@indeedemail.com3478630020Work ExperienceInternNew York Stage Supreme Court - Brooklyn, NYJuly 2019 to August 2019Observed court proceeding..."
331,Yayonah Bangura,YAYONAH BANGURAPage Two of TwoYAYONAH BANGURA (732) 421-4186 Yayonahb@gmail.com www.linkedin.com/in/yayonahbWebsite: theaknown.com/features-1PROFILEAmbitious and motivated communications special...
332,Zachary Brooks,"Zachary H. Brooks 2855 Nicole Ct., Oceanside, New York, 11572 516-761-8168 | Zachary.Brooks@quinnipiac.edu EDUCATION QUINNIPIAC UNIVERSITY | School of Communications ..."


In [69]:
# Resort the names and reset the index

resumes = resumes.sort_values('username').reset_index(drop=True)

In [70]:
# Merge the cover letter and resume dataframes, matching the common names

compiled_df = cover_letter.merge(resumes, left_on='username', right_on='username')

In [71]:
compiled_df

,username,title,raw_cover_letters,Unnamed: 0,raw_resume
0,Aakash Patel,Media Planner,"I was excited to discover the Media Planner position with Rookieplay, which I saw posted on Indeed. Rookieplay is a company on the rise, and as a recent college graduate with 2 years of diverse ma...",NaN,"Microsoft Word - Aakash Patel 1\tAakash Patel aakashp047@gmail.com v (516) 404-6119 v Syosset, NY EDUCATION Stony Brook University ..."
1,Aaron Neal,Media Planner,"Dear Hiring Manager,I am writing to you today to apply for the open position of Media Planner. As a marketing professional with 2+ years of experience I believe that I will be an asset to the Rook...",NaN,"_Aaron Neal_052020_v2A A R O N M . N E A LSTRATEGISTPERSONALTRAITS & SKILLSEDUCATIONCONTACT770.871.5294aaronm.neal3@gmail.comProficient in Simmons OneView, Global Web Index MS Office Perspective..."
2,Aaron Penzel,Media Planner,"Dear Hiring Manager,I am writing to express interest in Rookieplay’s Media Planner position. I am a creative and out-of-the-box marketer in the entertainment industry, with experience in both non-...",NaN,"AARON PENZELAARON PENZELENTERTAINMENT ADVERTISERCONTACTPHONE: (908) 217-7289 EMAIL: A.PENZEL724@GMAIL.COM 23-43 28TH STREET LONG ISLAND CITY, NY 11105 WORK EXPERIENCEManaged all traditional media..."
3,Abigail Nelson,NaN,"White and Pastel Teal Minimalist Resume A B I G A I L A N N A L I S E N E L S O N(651) 408-2139 | abigail.nel@yahoo.com | 10849 Andes Circle, IGH, MN 55077TO:RookieplayNew York, NYFROM:Abigail N...",0.0,"White and Pastel Teal Minimalist A B I G A I L N E L S O N (651) 408-2139 | aanelson@iastate.edu | 10849 Andes Circle, IGH, MN 55077Portfolio: https://aanelson.wixsite.com/abigailnelsonLinkedIn:..."
4,Abraham Jauregui,Media Planner,"June 7th, 2020RookieplayNew York, New YorkDear Hiring Professional,Please accept this letter as an expression of my interest in the Media Planner position. I am a highly motivated and progress-foc...",NaN,"Abraham JaureguiAbraham Jauregui220-20 73rd Ave, Oakland Gardens NY 11364Cell: 917-561-9640 - abj997@aol.comSkills \t\t· Marketing and public relations; social media management· Fluent in Spani..."
...,...,...,...,...,...
110,Vickent Vera,Media Planner,"Vickent VeraNew York, NY(917)3918411vickentvera@gmail.comGreetings,I’ve been exposed and nurtured in movies and the internet from a very young age. Creating content like that has been my primary c...",NaN,"Vickent VeraVideo Production | Video Editor | PhotographerNew York, NYvickentvera7_2tf@indeedemail.com(917) 391-8411I was born in Queens, NY. Both my parents come from the fourth most populated c..."
111,Vincent Gray,Media Planner,"Dear Hiring Manager:Please accept this letter and the accompanying resume as my expressed interest in the position of Media Planner currently available at your organization.As my resume reflects, ...",NaN,"Vincent GRay's Client NamePage | 2Vincent Grayvinnygray40@gmail.com • Piscataway, New Jersey 08854 • (732) 309-7075\t\tMarketing Professional\t\t\t\tInnovative, self-motivated, and tech-savvy ind..."
112,Wania Nawaz,Media Planner,"I was motivated to apply to this internship because throughout my academic years I have fallen in love with marketing, specifically in social media marketing. My goal in the future would be workin...",NaN,"Microsoft Word - WANIA NAWAZWANIA NAWAZ Wnnawaz@gmail.com | (516) 473-9496 | Farmingdale, NY 11735 Education SUNY Farmingdale | Farmingdale, NY | Aug 2017 - May 2020 Bachelor of Science: Business..."
113,William Day,Media Planner,My name is William Day and I am writing to express my greatest interest in becoming a member of the Rookeiplay team. I hope that you will see that my combined experience in multiple facets of the ...,NaN,"William DayThe ""Jack of all Trades"" When it Comes to MediaMiami Beach, FL 33140williamday846_ys9@indeedemail.com770-687-4809It may be unorthodox to say this but I feel that to best understand who..."


In [73]:
resumes_usernames = resumes['username']
resumes_raw_resume = resumes['raw_resume']

In [74]:
to_add_usernames = []
to_add_resumes = []

for username in resumes_usernames:
    if username not in list(compiled_df['username']):
        to_add_usernames.append(username)
for resume in resumes_raw_resume:
    if resume not in list(compiled_df['raw_resume']):
        to_add_resumes.append(resume)

In [75]:
len(to_add_usernames)

219

In [76]:
len(to_add_resumes)

219

In [77]:
to_add_df = pd.DataFrame(list(zip(to_add_usernames, to_add_resumes)), 
               columns =['username', 'raw_resume']) 

In [78]:
to_add_df['title']='Media Planner'

In [79]:
to_add_df

,username,raw_resume,title
0,Abby Cleary,"Abby ClearyBuffalo, NYabbycleary7_pb3@indeedemail.com(716) 238-0440Authorized to work in the US for any employerWork ExperienceInternSwanson Communications - New York, NYJune 2019 to Present• Ass...",Media Planner
1,Abisola Sanya,Abiola AbisolaSanyaB u s i n e s s D e v e l o p m e n t Business Development & Research Business DevelopmentCommunicated and worked actively with hotel investment and risk assessme...,Media Planner
2,Abner Barriera,"Abner BarrieraRutgers University: School of Communication and Information GraduateBloomfield, NJabarriera10969_zna@indeedemail.com(302)887-6786Journalism/Media: • Studied the foundations of jour...",Media Planner
3,Abra Burkett,"Abra BurkettBrooklyn, NY 11206abraburkett3_9qo@indeedemail.com(310) 351-6405Authorized to work in the US for any employerWork ExperienceSeasonalSix Flags America - Bowie, MDMarch 2019 to PresentS...",Media Planner
4,Addy Fadina,"ADDYFADINANew York, NY | 917- 569- 7156 | AFadina2011@gmail.com |Linkedin.com/in/AddyFadinaSUMMARY OF QUALIFICATIONS ...",Media Planner
...,...,...,...
214,Vincent Rullo,"Vincent Rullo 514 Bay Ridge Parkway Apt #2BBrooklyn, NY 11209Email: vinnierullo27@gmail.comPhone : 518-596-8359EMPLOYMENT HISTORYThe Topps Company ...",Media Planner
215,Waid Harbison,"Waid HarbisonCullman, AL 35055waid85_atc@indeedemail.com256-708-1650Work ExperiencePresident & Chief Marketing OfficerA-1 Marketing Services LLCApril 2019 to PresentStarted own business working w...",Media Planner
216,Yasmina Ouedraogo,"Yasmina OuedraogoBrooklyn, NY 11207yasminaouedraogo9_k5n@indeedemail.com3478630020Work ExperienceInternNew York Stage Supreme Court - Brooklyn, NYJuly 2019 to August 2019Observed court proceeding...",Media Planner
217,Yayonah Bangura,YAYONAH BANGURAPage Two of TwoYAYONAH BANGURA (732) 421-4186 Yayonahb@gmail.com www.linkedin.com/in/yayonahbWebsite: theaknown.com/features-1PROFILEAmbitious and motivated communications special...,Media Planner


In [80]:
for row in to_add_df:
    compiled_df['username'].append(to_add_df.username)
    compiled_df['title'].append(to_add_df.title)
    compiled_df['raw_cover_letters'].append(None)
    compiled_df['raw_resume'].append(to_add_df.raw_resume)
    

In [81]:
compiled_df

,username,title,raw_cover_letters,Unnamed: 0,raw_resume
0,Aakash Patel,Media Planner,"I was excited to discover the Media Planner position with Rookieplay, which I saw posted on Indeed. Rookieplay is a company on the rise, and as a recent college graduate with 2 years of diverse ma...",NaN,"Microsoft Word - Aakash Patel 1\tAakash Patel aakashp047@gmail.com v (516) 404-6119 v Syosset, NY EDUCATION Stony Brook University ..."
1,Aaron Neal,Media Planner,"Dear Hiring Manager,I am writing to you today to apply for the open position of Media Planner. As a marketing professional with 2+ years of experience I believe that I will be an asset to the Rook...",NaN,"_Aaron Neal_052020_v2A A R O N M . N E A LSTRATEGISTPERSONALTRAITS & SKILLSEDUCATIONCONTACT770.871.5294aaronm.neal3@gmail.comProficient in Simmons OneView, Global Web Index MS Office Perspective..."
2,Aaron Penzel,Media Planner,"Dear Hiring Manager,I am writing to express interest in Rookieplay’s Media Planner position. I am a creative and out-of-the-box marketer in the entertainment industry, with experience in both non-...",NaN,"AARON PENZELAARON PENZELENTERTAINMENT ADVERTISERCONTACTPHONE: (908) 217-7289 EMAIL: A.PENZEL724@GMAIL.COM 23-43 28TH STREET LONG ISLAND CITY, NY 11105 WORK EXPERIENCEManaged all traditional media..."
3,Abigail Nelson,NaN,"White and Pastel Teal Minimalist Resume A B I G A I L A N N A L I S E N E L S O N(651) 408-2139 | abigail.nel@yahoo.com | 10849 Andes Circle, IGH, MN 55077TO:RookieplayNew York, NYFROM:Abigail N...",0.0,"White and Pastel Teal Minimalist A B I G A I L N E L S O N (651) 408-2139 | aanelson@iastate.edu | 10849 Andes Circle, IGH, MN 55077Portfolio: https://aanelson.wixsite.com/abigailnelsonLinkedIn:..."
4,Abraham Jauregui,Media Planner,"June 7th, 2020RookieplayNew York, New YorkDear Hiring Professional,Please accept this letter as an expression of my interest in the Media Planner position. I am a highly motivated and progress-foc...",NaN,"Abraham JaureguiAbraham Jauregui220-20 73rd Ave, Oakland Gardens NY 11364Cell: 917-561-9640 - abj997@aol.comSkills \t\t· Marketing and public relations; social media management· Fluent in Spani..."
...,...,...,...,...,...
110,Vickent Vera,Media Planner,"Vickent VeraNew York, NY(917)3918411vickentvera@gmail.comGreetings,I’ve been exposed and nurtured in movies and the internet from a very young age. Creating content like that has been my primary c...",NaN,"Vickent VeraVideo Production | Video Editor | PhotographerNew York, NYvickentvera7_2tf@indeedemail.com(917) 391-8411I was born in Queens, NY. Both my parents come from the fourth most populated c..."
111,Vincent Gray,Media Planner,"Dear Hiring Manager:Please accept this letter and the accompanying resume as my expressed interest in the position of Media Planner currently available at your organization.As my resume reflects, ...",NaN,"Vincent GRay's Client NamePage | 2Vincent Grayvinnygray40@gmail.com • Piscataway, New Jersey 08854 • (732) 309-7075\t\tMarketing Professional\t\t\t\tInnovative, self-motivated, and tech-savvy ind..."
112,Wania Nawaz,Media Planner,"I was motivated to apply to this internship because throughout my academic years I have fallen in love with marketing, specifically in social media marketing. My goal in the future would be workin...",NaN,"Microsoft Word - WANIA NAWAZWANIA NAWAZ Wnnawaz@gmail.com | (516) 473-9496 | Farmingdale, NY 11735 Education SUNY Farmingdale | Farmingdale, NY | Aug 2017 - May 2020 Bachelor of Science: Business..."
113,William Day,Media Planner,My name is William Day and I am writing to express my greatest interest in becoming a member of the Rookeiplay team. I hope that you will see that my combined experience in multiple facets of the ...,NaN,"William DayThe ""Jack of all Trades"" When it Comes to MediaMiami Beach, FL 33140williamday846_ys9@indeedemail.com770-687-4809It may be unorthodox to say this but I feel that to best understand who..."


In [82]:
compiled_df = compiled_df.append(to_add_df)

In [83]:
compiled_df = compiled_df.reset_index(drop=True)

In [84]:
compiled_df

,username,title,raw_cover_letters,Unnamed: 0,raw_resume
0,Aakash Patel,Media Planner,"I was excited to discover the Media Planner position with Rookieplay, which I saw posted on Indeed. Rookieplay is a company on the rise, and as a recent college graduate with 2 years of diverse ma...",NaN,"Microsoft Word - Aakash Patel 1\tAakash Patel aakashp047@gmail.com v (516) 404-6119 v Syosset, NY EDUCATION Stony Brook University ..."
1,Aaron Neal,Media Planner,"Dear Hiring Manager,I am writing to you today to apply for the open position of Media Planner. As a marketing professional with 2+ years of experience I believe that I will be an asset to the Rook...",NaN,"_Aaron Neal_052020_v2A A R O N M . N E A LSTRATEGISTPERSONALTRAITS & SKILLSEDUCATIONCONTACT770.871.5294aaronm.neal3@gmail.comProficient in Simmons OneView, Global Web Index MS Office Perspective..."
2,Aaron Penzel,Media Planner,"Dear Hiring Manager,I am writing to express interest in Rookieplay’s Media Planner position. I am a creative and out-of-the-box marketer in the entertainment industry, with experience in both non-...",NaN,"AARON PENZELAARON PENZELENTERTAINMENT ADVERTISERCONTACTPHONE: (908) 217-7289 EMAIL: A.PENZEL724@GMAIL.COM 23-43 28TH STREET LONG ISLAND CITY, NY 11105 WORK EXPERIENCEManaged all traditional media..."
3,Abigail Nelson,NaN,"White and Pastel Teal Minimalist Resume A B I G A I L A N N A L I S E N E L S O N(651) 408-2139 | abigail.nel@yahoo.com | 10849 Andes Circle, IGH, MN 55077TO:RookieplayNew York, NYFROM:Abigail N...",0.0,"White and Pastel Teal Minimalist A B I G A I L N E L S O N (651) 408-2139 | aanelson@iastate.edu | 10849 Andes Circle, IGH, MN 55077Portfolio: https://aanelson.wixsite.com/abigailnelsonLinkedIn:..."
4,Abraham Jauregui,Media Planner,"June 7th, 2020RookieplayNew York, New YorkDear Hiring Professional,Please accept this letter as an expression of my interest in the Media Planner position. I am a highly motivated and progress-foc...",NaN,"Abraham JaureguiAbraham Jauregui220-20 73rd Ave, Oakland Gardens NY 11364Cell: 917-561-9640 - abj997@aol.comSkills \t\t· Marketing and public relations; social media management· Fluent in Spani..."
...,...,...,...,...,...
329,Vincent Rullo,Media Planner,NaN,NaN,"Vincent Rullo 514 Bay Ridge Parkway Apt #2BBrooklyn, NY 11209Email: vinnierullo27@gmail.comPhone : 518-596-8359EMPLOYMENT HISTORYThe Topps Company ..."
330,Waid Harbison,Media Planner,NaN,NaN,"Waid HarbisonCullman, AL 35055waid85_atc@indeedemail.com256-708-1650Work ExperiencePresident & Chief Marketing OfficerA-1 Marketing Services LLCApril 2019 to PresentStarted own business working w..."
331,Yasmina Ouedraogo,Media Planner,NaN,NaN,"Yasmina OuedraogoBrooklyn, NY 11207yasminaouedraogo9_k5n@indeedemail.com3478630020Work ExperienceInternNew York Stage Supreme Court - Brooklyn, NYJuly 2019 to August 2019Observed court proceeding..."
332,Yayonah Bangura,Media Planner,NaN,NaN,YAYONAH BANGURAPage Two of TwoYAYONAH BANGURA (732) 421-4186 Yayonahb@gmail.com www.linkedin.com/in/yayonahbWebsite: theaknown.com/features-1PROFILEAmbitious and motivated communications special...


In [89]:
# Drops extra columns. We only want username and text columns
compiled_df = compiled_df.drop(compiled_df.columns[3], axis=1)

In [90]:
final_df = pd.read_csv('data/raw_data.csv', index_col=0)

In [91]:
final_df

,username,title,raw_cover_letters,raw_resume
0,Abdul Albab,Accounts Payable Specialist,NaN,"Abdul AlbabBronx, NY 10467abdulalbab2_n6u@indeedemail.com(347) 615-8170Work ExperienceAccountantCS Corporation - Bronx, NYJune 2017 to Present• Manage and oversee daily Accounts Payable and Accou..."
1,Abdul Shafaq,Accounts Payable Specialist,NaN,"Abdul Waheed Shafaq8581 N. Monroe Avenue, Lindenhurst, NY, 11757 | shafaqw2014@gmail.com | 347-335-8422Skills and Qualifications· Professional interpreter & translator; fluent in English, Dari, Pa..."
2,Abigail Riveron,Accounts Payable Specialist,"Dear Hiring Manager,I would like to express my deep interest in a position in your company.With my strong understanding of business and accounting principles gained throughout my education backgro...","CollegeGrad.com TemplateABIGAIL RIVERONMiami, FL 33177Mobile: 305.746.1662ariveron00@gmail.comProfessional SummaryEfficient, accurate and detail-oriented who takes initiative and has an innate dr..."
3,Abirami Suresh,Accounts Payable Specialist,NaN,"- Abirami Suresh.pagesAbirami Suresh 17 Bethel Lane, Schaumburg IL 60194 Mobile: 630-379-1405 Email: abisuresh62@gmail.com CAREER SUMMARY Highly self-motivated Senior Auditing Accountant with dem..."
4,Abria Bell,Training Manager,NaN,"Abria BellProject AdministratorChicago, IL 60617bellabria1213_2io@indeedemail.com7739700598#readytoworkWork ExperienceBilling / Administrative AssistantProject Org and Design Studio Inc - Chicago..."
...,...,...,...,...
2120,Zichen Nie,Accounts Payable Specialist,NaN,"ZICHEN (CONSTANCE) NIEHacienda Heightsznie003@ucr.edu951-213-1736Professional Auditing, Systems, Managerial, Financial Statement Analysis, Fraud and Forensic Accounting, ITSkills..."
2121,Zihao Zhou,Accounts Payable Specialist,NaN,"Zihao ZhouManager / Business Development DirectorBoston, MALike challenging new tasks. Positive personality. Good at communication, experienced marketing andproducts specialist.Willing to relocat..."
2122,Zira Cortes,Training Manager,NaN,"Zira CortesSubstitute Teacher - SANDRIDGE Elementary SchoolChicago, IL 60633ziracortes9_rew@indeedemail.com(773)-704-4172I am seeking a full-time position where I can use my experience and educat..."
2123,Zoya Buyankina,Senior Account Manager,NaN,"ZOYA BUYANKINA SUMMARY: A young, self-motivated and achievement-oriented Business Development Analyst with more than 3 years of experience in the business development process and different marketi..."


In [92]:
final_df = final_df.append(compiled_df)

In [93]:
final_df.to_csv('data/raw_data.csv')

## Check for duplicates

In [42]:
final_df = pd.read_csv(r'C:\Users\sambe\Projects\Cover_Letter_Analysis\data\raw_data.csv', index_col=0)

In [43]:
final_df = final_df.sort_values('username').reset_index(drop=True)

In [45]:
final_df.to_csv('data/raw_data.csv')

In [94]:
final_df

,username,title,raw_cover_letters,raw_resume
0,Abdul Albab,Accounts Payable Specialist,NaN,"Abdul AlbabBronx, NY 10467abdulalbab2_n6u@indeedemail.com(347) 615-8170Work ExperienceAccountantCS Corporation - Bronx, NYJune 2017 to Present• Manage and oversee daily Accounts Payable and Accou..."
1,Abdul Shafaq,Accounts Payable Specialist,NaN,"Abdul Waheed Shafaq8581 N. Monroe Avenue, Lindenhurst, NY, 11757 | shafaqw2014@gmail.com | 347-335-8422Skills and Qualifications· Professional interpreter & translator; fluent in English, Dari, Pa..."
2,Abigail Riveron,Accounts Payable Specialist,"Dear Hiring Manager,I would like to express my deep interest in a position in your company.With my strong understanding of business and accounting principles gained throughout my education backgro...","CollegeGrad.com TemplateABIGAIL RIVERONMiami, FL 33177Mobile: 305.746.1662ariveron00@gmail.comProfessional SummaryEfficient, accurate and detail-oriented who takes initiative and has an innate dr..."
3,Abirami Suresh,Accounts Payable Specialist,NaN,"- Abirami Suresh.pagesAbirami Suresh 17 Bethel Lane, Schaumburg IL 60194 Mobile: 630-379-1405 Email: abisuresh62@gmail.com CAREER SUMMARY Highly self-motivated Senior Auditing Accountant with dem..."
4,Abria Bell,Training Manager,NaN,"Abria BellProject AdministratorChicago, IL 60617bellabria1213_2io@indeedemail.com7739700598#readytoworkWork ExperienceBilling / Administrative AssistantProject Org and Design Studio Inc - Chicago..."
...,...,...,...,...
329,Vincent Rullo,Media Planner,NaN,"Vincent Rullo 514 Bay Ridge Parkway Apt #2BBrooklyn, NY 11209Email: vinnierullo27@gmail.comPhone : 518-596-8359EMPLOYMENT HISTORYThe Topps Company ..."
330,Waid Harbison,Media Planner,NaN,"Waid HarbisonCullman, AL 35055waid85_atc@indeedemail.com256-708-1650Work ExperiencePresident & Chief Marketing OfficerA-1 Marketing Services LLCApril 2019 to PresentStarted own business working w..."
331,Yasmina Ouedraogo,Media Planner,NaN,"Yasmina OuedraogoBrooklyn, NY 11207yasminaouedraogo9_k5n@indeedemail.com3478630020Work ExperienceInternNew York Stage Supreme Court - Brooklyn, NYJuly 2019 to August 2019Observed court proceeding..."
332,Yayonah Bangura,Media Planner,NaN,YAYONAH BANGURAPage Two of TwoYAYONAH BANGURA (732) 421-4186 Yayonahb@gmail.com www.linkedin.com/in/yayonahbWebsite: theaknown.com/features-1PROFILEAmbitious and motivated communications special...


In [33]:
test_df = final_df

In [34]:
test_df

,username,title,raw_cover_letters,raw_resume
0,Abdul Albab,Accounts Payable Specialist,NaN,"Abdul AlbabBronx, NY 10467abdulalbab2_n6u@indeedemail.com(347) 615-8170Work ExperienceAccountantCS Corporation - Bronx, NYJune 2017 to Present• Manage and oversee daily Accounts Payable and Accou..."
1,Abdul Shafaq,Accounts Payable Specialist,NaN,"Abdul Waheed Shafaq8581 N. Monroe Avenue, Lindenhurst, NY, 11757 | shafaqw2014@gmail.com | 347-335-8422Skills and Qualifications· Professional interpreter & translator; fluent in English, Dari, Pa..."
2,Abigail Riveron,Accounts Payable Specialist,"Dear Hiring Manager,I would like to express my deep interest in a position in your company.With my strong understanding of business and accounting principles gained throughout my education backgro...","CollegeGrad.com TemplateABIGAIL RIVERONMiami, FL 33177Mobile: 305.746.1662ariveron00@gmail.comProfessional SummaryEfficient, accurate and detail-oriented who takes initiative and has an innate dr..."
3,Abirami Suresh,Accounts Payable Specialist,NaN,"- Abirami Suresh.pagesAbirami Suresh 17 Bethel Lane, Schaumburg IL 60194 Mobile: 630-379-1405 Email: abisuresh62@gmail.com CAREER SUMMARY Highly self-motivated Senior Auditing Accountant with dem..."
4,Abria Bell,Training Manager,NaN,"Abria BellProject AdministratorChicago, IL 60617bellabria1213_2io@indeedemail.com7739700598#readytoworkWork ExperienceBilling / Administrative AssistantProject Org and Design Studio Inc - Chicago..."
...,...,...,...,...
2450,Zira Cortes,Training Manager,NaN,"Zira CortesSubstitute Teacher - SANDRIDGE Elementary SchoolChicago, IL 60633ziracortes9_rew@indeedemail.com(773)-704-4172I am seeking a full-time position where I can use my experience and educat..."
2451,Zoya Buyankina,Senior Account Manager,NaN,"ZOYA BUYANKINA SUMMARY: A young, self-motivated and achievement-oriented Business Development Analyst with more than 3 years of experience in the business development process and different marketi..."
2453,Zughem Tovar-Riezgo,Accounts Payable Specialist,NaN,"Zughem Tovar-Riezgo 40 Westwood Circle Roslyn Heights, NY 11577 (347) 581-4703 priezgo@gmail.com EXPERIENCE Rinapher Inc, ​Astoria, NY — ​Administrative Assistant /Bookkeeper​ ​June 2015 - Febr..."
2454,for Corey Rosenberg C D,Accounts Payable Specialist,NaN,CHECK OUT MY WORKhttp://coreyrosenberg.com


In [22]:
test_df['username'][27]

'Alahree Martin'

In [31]:
test_df.drop_duplicates(subset ="username", 
                     keep = 'first', inplace = True) 
      

In [32]:
test_df

,username,title,raw_cover_letters,raw_resume
0,Abdul Albab,Accounts Payable Specialist,NaN,"Abdul AlbabBronx, NY 10467abdulalbab2_n6u@indeedemail.com(347) 615-8170Work ExperienceAccountantCS Corporation - Bronx, NYJune 2017 to Present• Manage and oversee daily Accounts Payable and Accou..."
1,Abdul Shafaq,Accounts Payable Specialist,NaN,"Abdul Waheed Shafaq8581 N. Monroe Avenue, Lindenhurst, NY, 11757 | shafaqw2014@gmail.com | 347-335-8422Skills and Qualifications· Professional interpreter & translator; fluent in English, Dari, Pa..."
2,Abigail Riveron,Accounts Payable Specialist,"Dear Hiring Manager,I would like to express my deep interest in a position in your company.With my strong understanding of business and accounting principles gained throughout my education backgro...","CollegeGrad.com TemplateABIGAIL RIVERONMiami, FL 33177Mobile: 305.746.1662ariveron00@gmail.comProfessional SummaryEfficient, accurate and detail-oriented who takes initiative and has an innate dr..."
3,Abirami Suresh,Accounts Payable Specialist,NaN,"- Abirami Suresh.pagesAbirami Suresh 17 Bethel Lane, Schaumburg IL 60194 Mobile: 630-379-1405 Email: abisuresh62@gmail.com CAREER SUMMARY Highly self-motivated Senior Auditing Accountant with dem..."
4,Abria Bell,Training Manager,NaN,"Abria BellProject AdministratorChicago, IL 60617bellabria1213_2io@indeedemail.com7739700598#readytoworkWork ExperienceBilling / Administrative AssistantProject Org and Design Studio Inc - Chicago..."
...,...,...,...,...
2450,Zira Cortes,Training Manager,NaN,"Zira CortesSubstitute Teacher - SANDRIDGE Elementary SchoolChicago, IL 60633ziracortes9_rew@indeedemail.com(773)-704-4172I am seeking a full-time position where I can use my experience and educat..."
2451,Zoya Buyankina,Senior Account Manager,NaN,"ZOYA BUYANKINA SUMMARY: A young, self-motivated and achievement-oriented Business Development Analyst with more than 3 years of experience in the business development process and different marketi..."
2453,Zughem Tovar-Riezgo,Accounts Payable Specialist,NaN,"Zughem Tovar-Riezgo 40 Westwood Circle Roslyn Heights, NY 11577 (347) 581-4703 priezgo@gmail.com EXPERIENCE Rinapher Inc, ​Astoria, NY — ​Administrative Assistant /Bookkeeper​ ​June 2015 - Febr..."
2454,for Corey Rosenberg C D,Accounts Payable Specialist,NaN,CHECK OUT MY WORKhttp://coreyrosenberg.com


In [ ]:
baby_test_df = 